In [ ]:
!apt-get update
!apt-get install -y tesseract-ocr libtesseract-dev

!pip install -U torch torchvision

!pip install -U cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git'
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .
%cd ..

!pip install -U matplotlib numpy opencv-python-headless pandas tqdm tabulate gradio streamlit huggingface_hub pytesseract

import torch
import torchvision
import detectron2
#import encapsu_view
import gradio as gr
import cv2
import matplotlib.pyplot as plt
import pytesseract

print("PyTorch:", torch.__version__)
print("Detectron2:", detectron2.__version__)

In [ ]:
!pip install git+https://github.com/MaksymTymkovych/alginate-capsules-analysis


In [ ]:
import zipfile, json, io, os, shutil, numpy as np
import cv2
import matplotlib.pyplot as plt

from encapsu_view.analysis.detection import process_image_with_capsules, capsule_detector, inner_detector
from encapsu_view.analysis.scale import scale_detector
from encapsu_view.visualization.hierarchy import visualize_tree, plot_capsule_hierarchy

from encapsu_view.analysis.scale import create_color_range_from_bgr, detect_scale_ruller, detect_lines_from_mask, find_longest_horizontal_line, extract_text_from_ruler_region


def load_image_cv(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError(f"Failed to read image: {file_path}")

    if len(img.shape) == 2:
        if img.dtype == np.uint16:
            img = (img / 256).astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        return img

    if img.shape[2] == 3:
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    if img.shape[2] == 4:
        return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

    raise ValueError(f"Unsupported shape: {img.shape}")

def run_capsule_analysis(image_path):
    img_rgb = load_image_cv(image_path)

    plt.figure(figsize=(8, 6))
    plt.imshow(img_rgb)
    plt.axis('off')
    plt.title("Loaded Image")
    plt.show()

    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    ruler_color = (0, 0, 255) # BGR
    ruler_color_range = create_color_range_from_bgr(ruler_color, tolerance=100)

    image = img_bgr
    color_ranges = ruler_color_range
    min_line_length: int = 100,
    text_region_height: int = 150

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    for color_range in color_ranges:
        lower = np.array(color_range['lower'])
        upper = np.array(color_range['upper'])
        print(lower)
        print(upper)
        range_mask = cv2.inRange(hsv, lower, upper)
        mask = cv2.bitwise_or(mask, range_mask)

    plt.imshow(mask)
    plt.show()

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    lines = detect_lines_from_mask(mask, min_line_length)
    
    ruler_line = find_longest_horizontal_line(lines, min_line_length)
    
    text_info = extract_text_from_ruler_region(image, mask, ruler_line, text_region_height)


    detected_data = process_image_with_capsules(img_bgr, capsule_detector, inner_detector, scale_detector)
    processed_data = detected_data.convert_to_processed_data(ellipse_method="direct")

    raw_vis = cv2.cvtColor(detected_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)
    processed_vis = cv2.cvtColor(processed_data.visualize_detection(img_bgr), cv2.COLOR_BGR2RGB)

    print("\n--- Capsule Tree ---")
    visualize_tree(processed_data)

    json_data = processed_data.to_dict()

    plot_capsule_hierarchy(processed_data)

    return raw_vis, processed_vis, json_data


def process_batch(zip_file_path):
    temp_in = "temp_batch"
    temp_out = "temp_batch_out"
    for p in [temp_in, temp_out]:
        if os.path.exists(p):
            shutil.rmtree(p)
        os.makedirs(p)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(temp_in)

    results = {}
    output_images = []

    for fname in os.listdir(temp_in):
        if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".tiff")):
            continue
        path = os.path.join(temp_in, fname)
        img = cv2.imread(path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 80, 180)

        out_img = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)
        out_path = os.path.join(temp_out, fname)
        cv2.imwrite(out_path, cv2.cvtColor(out_img, cv2.COLOR_RGB2BGR))
        output_images.append(out_path)

        results[fname] = {"shape": list(img.shape), "edges_detected": int(np.sum(edges > 0))}

    zip_out_path = "processed_out.zip"
    with zipfile.ZipFile(zip_out_path, 'w') as zipf:
        for f in os.listdir(temp_out):
            zipf.write(os.path.join(temp_out, f), arcname=f)

    return output_images, results, zip_out_path


if __name__ == "__main__":
    image_path = "./Snap-21062.tif"  
    raw_vis, processed_vis, json_data = run_capsule_analysis(image_path)
    print("\nProcessed JSON:")
    print(json.dumps(json_data, indent=2))

    zip_path = "test_images.zip"  
    imgs, batch_res, zip_out = process_batch(zip_path)
    print("\nBatch results JSON:")
    print(json.dumps(batch_res, indent=2))
    print("Processed ZIP:", zip_out)
